In [2]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
from __future__ import print_function # so print doesn't show brackets

sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo
import DataBase 
import QMD
import QML
import ModelGeneration
import BayesF
import matplotlib.pyplot as plt

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

warnings.filterwarnings("ignore", message='Negative weights occured', category=RuntimeWarning)



class NewQMDClass():
    #TODO: rename as ModelsDevelopmentClass when finished
    def __init__(self,
                 initial_op_list,
                 true_operator='x',
                 maxmodnum=3, 
                 max_dimension=4, 
                 gaussian=True,
                 max_num_layers = 10,
                 max_num_branches = 20 
                ):
#    def __init__(self, initial_op_list, true_op_list, true_param_list):
        
        trueOp = DataBase.operator(true_operator)
        self.TrueOpName = true_operator
        self.MaxDimension = max_dimension
        self.TrueOpDim = trueOp.num_qubits
        self.InitialOpList = initial_op_list
        self.TrueOpList = trueOp.constituents_operators
        self.TrueParamsList = [random.random() for i in self.TrueOpList] # TODO: actual true params?
        #self.TrueHam = evo.getH(self.TrueParamsList, self.TrueOpList)
        self.TrueHam = np.tensordot(self.TrueParamsList, self.TrueOpList, axes=1)
        self.MaxModNum = maxmodnum #TODO: necessary?
        self.gaussian = gaussian
        self.NumModels = len(initial_op_list)
        self.NumParticles = 1000
        self.MaxBranchID = max_num_branches
        self.HighestBranchID = 0
        self.MaxLayerNumber = max_num_layers
        self.BranchChampions = {}
        self.LayerChampions = {}
        self.InterBranchChampions = {}
        self.GlobalEpoch = 0 
        
#        for i in range(self.MaxBranchID+1):
#            self.BranchChampions[i] = 0
        for i in range(self.MaxLayerNumber+1):
            self.LayerChampions[i] = 0
        
        print("\nQMD has true operator: ", true_operator)
        print("QMD has true params : ", self.TrueParamsList, "\n")

        # Initialise database and lists.
        self.initiateDB()
        
    def initiateDB(self):
        
        ## TODO: Models should be initialised with appropriate TrueOp dimension -- see getListTrueOpByDimension function
        self.db, self.legacy_db, self.model_lists = \
            DataBase.launch_db(
                true_op_name = self.TrueOpName,
                gen_list = self.InitialOpList,
                true_ops = self.TrueOpList,
                true_params = self.TrueParamsList
            )
    def addModel(self, model, treeID=0):
        self.NumModels += int(1)
        tryAddModel = DataBase.add_model(
            model_name = model,
            running_database = self.db,
            num_particles = self.NumParticles, 
            true_op_name = self.TrueOpName,
            model_lists = self.model_lists,
            true_ops = self.TrueOpList,
            true_params = self.TrueParamsList,
            treeID = treeID,
            modelID = int(self.NumModels),
        )
        
        if tryAddModel is False: 
            self.NumModels-=int(1)

    def newBranch(self, model_list):
        self.HighestBranchID += 1
        branchID = self.HighestBranchID
        for model in model_list:
            self.addModel(model, treeID=branchID)
            
    def getModelInstance(self, name):
        try: 
            instance = DataBase.get_qml_instance(self.db, name)
            return instance
        except: 
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Model not found.")
    def getOperatorInstance(self, name):
        try: 
            return DataBase.get_operator_instance(self.db, name)
        except:
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Operator not found.")

    def getModelDBIndex(self, name):
        return DataBase.get_location(self.db, name)

    def getModelInstanceFromID(self, model_id):
        return DataBase.model_instance_from_id(self.db, model_id)    
    
    def killModel(self, name):
        if name not in list(self.db['<Name>']):
            print("Cannot remove ", name, "; not in ", list(self.db["<Name>"]))
        else:
            print("Killing model", name)
            # Add to legacy_db
            DataBase.move_to_legacy(self.db, self.legacy_db, name)
            model_instance = self.getModelInstance(name)
            operator_instance = self.getOperatorInstance(name)
            # Remove from self.db
            self.db = DataBase.remove_model(self.db, name)
            del model_instance
            del operator_instance
    
            #TODO: plot?

    def runIQLE(self, model, num_exp=50):
        model_exists=False
        if model in list(self.db['<Name>']):
            model_exists = True
        elif model in list(self.legacy_db['<Name>']):
            print("Model ", model, " previously considered and retired.")
            
        if model_exists: 
            model_instance = self.getModelInstance(model)
            model_instance.UpdateModel(num_exp)
            #model_instance.BayesOnModelsWithinTrees
        else: 
            print("Model ", model ,"does not exist")

    def runAllActiveModelsIQLE(self, num_exp):
        active_models = self.db.loc[self.db['Status']=='Active']['<Name>']

        for model in active_models:
            self.runIQLE(model=model, num_exp=num_exp)
            
        
    def updateModelRecord(self, name, field, new_value=None, increment=None):
        DataBase.update_field(
            self.db, 
            name,
            field,
            new_value=new_value,
            increment=increment
        )
    def pullField(self, name, field):
        return DataBase.pull_field(self.db, name, field)

    def statusChangeBranch(self, branchID, new_status='Saturated'):
        self.db.loc[ self.db['TreeID']==branchID , 'Status'] = new_status

    def statusChangeModel(self, model_name, new_status='Saturated'):
        self.db.loc[ self.db['<Name>']==model_name , 'Status'] = new_status
        
    def getListTrueOpByDimension(self):
        self.TrueOpListByDim = {}
        self.TrueParamByDim = {}
        for dim in range(1, 1+self.MaxDimension):
            new_op = ModelGeneration.identity_interact(subsystem=self.TrueOpName, num_qubits=dim, return_operator=True)
            self.TrueOpListByDim[dim] = new_op.constituents_operators
        for i in range(1, self.TrueOpDim+1):
            self.TrueParamByDim[i] = self.TrueParamsList
        for i in range(self.TrueOpDim+1, self.MaxDimension):
            self.TrueParamByDim[i] = [self.TrueParamsList[0]]

    def compareModels(self, log_comparison_high=200.0, num_times_to_use = 'all', model_a_id = None, model_b_id =None, model_a = None, model_b = None, name_a=None, name_b=None):
        # Either pass in name_a and name_b OR model_a and model_b
        if model_a is None and model_b is None:
            if model_a_id is not None and model_b_id is not None: 
                model_a = self.getModelInstanceFromID(model_a_id)
                model_b = self.getModelInstanceFromID(model_b_id)
            else: # if only names were passed 
                model_a = self.getModelInstance(name_a)
                model_b = self.getModelInstance(name_b)
        if model_a ==  model_b:
            print("Same models, don't need to compare.")
            return "Same Models"

        log_comparison_low = 1/log_comparison_high
        
        if num_times_to_use == 'all':
            times_a = model_a.TrackTime
        elif len(model_a.TrackTime) < num_times_to_use:
            times_a = model_a.TrackTime
        else: 
            times_a = model_a.TrackTime[num_times_to_use:]
        
        if num_times_to_use=='all':
            times_b = model_b.TrackTime
        elif len(model_b.TrackTime) < num_times_to_use:
            times_b = model_b.TrackTime
        else: 
            times_b = model_b.TrackTime[num_times_to_use:]
        gen_a = model_a.GenSimModel
        gen_b = model_b.GenSimModel

        times = times_a + times_b
        exps_a = get_exps(model_a, gen_a, times)
        exps_b = get_exps(model_b, gen_b, times)

        log_l_a =  get_log_likelihood(model_a, gen_a, exps_a)
        log_l_b =  get_log_likelihood(model_b, gen_b, exps_b)

        bayes_factor = np.exp(log_l_a - log_l_b)

        print("bayes_factor of log likelihoods between ", model_a.Name, " and ", model_b.Name, " = ", bayes_factor)
        if bayes_factor > log_comparison_high: 
            print("Point to ", model_a.Name)
            return "a"
        elif bayes_factor < log_comparison_low: 
            print("Point to ", model_b.Name)
            return "b"
        else:
            print("No real winner")            

            
    def compareModelsWithinBranch(self, treeID):
        active_models_in_tree = DataBase.active_model_ids_by_tree_id(qmd.db, treeID)
        
        models_points = {}
        for model_id in active_models_in_tree:
            models_points[model_id] = 0
        
        for i in range(len(active_models_in_tree)):
            mod1 = active_models_in_tree[i]
            for j in range(i, len(active_models_in_tree)): 
                mod2 = active_models_in_tree[j]
                res = self.compareModels(model_a_id = mod1, model_b_id=mod2)
                if res == "a":
                    models_points[mod1] += 1
                elif res == "b":
                    models_points[mod2] += 1
        champ_id = max(models_points, key=models_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        
        self.BranchChampions[int(treeID)] = champ_id
        
        print("Champion of tree ", treeID, " is ", champ_name)
        return models_points, champ_id

    
    def compareModelList(self, model_list, bayes_threshold = 50):
        models_points = {}
        for mod in model_list:
            models_points[mod] = 0
        
        for i in range(len(model_list)):
            mod1 = model_list[i]
            for j in range(i, len(model_list)):
                mod2 = model_list[j]
                if mod1 != mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=bayes_threshold)
                    if res == "a":
                        models_points[mod1] += 1
                    elif res == "b":
                        models_points[mod2]+=1

        champ_id = max(models_points, key=models_points.get)
        return champ_id
    
    def interBranchChampion(self, branch_list=[], global_champion=False):
        all_branches = self.db['TreeID'].unique()
        if global_champion == True: 
            trees = all_branches
        else: 
            trees = branch_list
        
        num_branches = len(trees)
        points_by_branches = [None] * num_branches
        champions_of_branches = [None] * num_branches

        for i in range(num_branches):
            treeID = trees[i]
            if treeID not in all_branches:
                print("Tree ID : ", treeID)
                warnings.warn("Tree not in database.")
                return False
            points_by_branches[i], champions_of_branches[i] = qmd.compareModelsWithinBranch(treeID)

        branch_champions_points = {}
        for c in champions_of_branches: 
            branch_champions_points[c] = 0

        for i in range(num_branches):
            mod1 = champions_of_branches[i]
            for j in range(i, num_branches):
                mod2 = champions_of_branches[j]
                if mod1!=mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=20.0)
                    if res == "a":
                        branch_champions_points[mod1] += 1
                    elif res == "b":
                        branch_champions_points[mod2]+=1
        self.ranked_champions = sorted(branch_champions_points, reverse=True)
        
        max_points = max(branch_champions_points.values())
        max_points_branches = [key for key, val in branch_champions_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1)
        else: 
            champ_id = max(branch_champions_points, key=branch_champions_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        
        branch_champ_names = [DataBase.model_name_from_id(self.db, mod_id) for mod_id in champions_of_branches]
        
        interBranchChampListID = len(self.InterBranchChampions)
        self.InterBranchChampions[interBranchChampListID] = [trees, [champ_id]]
        print("Champion of branches ", trees, " is ",  champ_name)
        return champ_name, branch_champ_names
    
    def globalChampionCalculation(self):
        trees = self.db['TreeID'].unique()
        
        num_trees = len(trees)
        self.points_by_trees = [None] * num_trees
        self.champions_of_trees = [None] * num_trees

        for i in range(num_trees):
            treeID = trees[i]
            self.points_by_trees[i], self.champions_of_trees[i] = qmd.compareModelsWithinBranch(treeID)

        self.champions_points = {}
        for c in self.champions_of_trees: 
            self.champions_points[c] = 0

        for i in range(num_trees):
            mod1 = self.champions_of_trees[i]
            for j in range(i, num_trees):
                mod2 = self.champions_of_trees[j]
                if mod1!=mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=10.0)
                    if res == "a":
                        self.champions_points[mod1] += 1
                    elif res == "b":
                        self.champions_points[mod2]+=1
        self.ranked_champions = sorted(self.champions_points, reverse=True)
        champ_id = max(self.champions_points, key=self.champions_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        print("Champion of Champions is",  champ_name)
    
    
def get_exps(model, gen, times):

    exps = np.empty(len(times), dtype=gen.expparams_dtype)
    exps['t'] = times

    for i in range(1, len(gen.expparams_dtype)):
        col_name = 'w_'+str(i)
        exps[col_name] = model.FinalParams[i-1,0] ## TODO: should be model.NewEval[i-1]???
    return exps

def get_log_likelihood(model, gen, exps):
    import copy
    updater = copy.deepcopy(model.Updater)
    data = gen.simulate_experiment(model.SimParams, exps)[0][0]
    updater.batch_update(data, exps, resample_interval=100)
    log_likelihood = np.sum(updater.log_total_likelihood)
    del updater
    return log_likelihood
            

    

In [3]:
import random
num_exp = 2

max_dim=2
dims = range(1,max_dim+1)
num_qubits=random.choice(dims)

true_op = 'xPy'
tree_3_ops = ['x', 'y', 'yTx', 'zPy'] # tree0
tree_2_ops = ['yTz', 'xPy']
tree_1_ops = ['z', 'zTi']
generators = ['xTi', 'xTy']
true_op = ModelGeneration.random_ising_chain(num_dimensions=num_qubits, num_directions = 1, include_identity=False)
#true_op = random.choice(generators)
print("True operator : ", true_op)

qmd= NewQMDClass(initial_op_list=generators, true_operator=true_op)
qmd.newBranch(tree_1_ops)
qmd.newBranch(tree_2_ops)
#qmd.newBranch(tree_3_ops)

qmd.runAllActiveModelsIQLE(num_exp=num_exp)
champ_id, branch_champ_names = qmd.interBranchChampion(global_champion=True)


for model in list(qmd.db['<Name>']):
    qmd.statusChangeModel(model, new_status='Saturated')

for champ in branch_champ_names:  
    # change these ones back (change to something like 'ActiveChampion' so it doesn't run IQLE again)
    qmd.statusChangeModel(champ, new_status='ReigningChampion')

new_models=ModelGeneration.new_model_list_grow_one_qubit(generator_list=branch_champ_names)
qmd.newBranch(new_models)

True operator :  zTz

QMD has true operator:  zTz
QMD has true params :  [0.9117574913518774] 

Model  xTi  not previously considered -- adding.
Initialization Ready
Model  xTy  not previously considered -- adding.
Initialization Ready
Before dimensionalising name. Name =  z true_dim =  2
Model  z  not previously considered -- adding.
Initialization Ready
Model  zTi  not previously considered -- adding.
Initialization Ready
Model  yTz  not previously considered -- adding.
Initialization Ready
Before dimensionalising name. Name =  xPy true_dim =  2
Model  xPy  not previously considered -- adding.
Initialization Ready
Initial time selected > 0.618212741801
Final time selected > 0.297484619975
Final Parameters mean and stdev:[ 0.47808425  0.57941404]
Initial time selected > 0.309892184209
Final time selected > 0.491078723446
Final Parameters mean and stdev:[ 0.46486943  0.56904327]
Initial time selected > 6.62194744228
Final time selected > 0.85872470606
Final Parameters mean and stdev:[ 

In [4]:
qmd.InterBranchChampions

{0: [array([ 0.,  1.,  2.]), [0.0]]}

In [5]:
qmd.BranchChampions

{0: 0.0, 1: 3.0, 2: 5.0}

In [6]:
qmd.db

,<Name>,Alph_Name,BayesSum,Dimensionalised_Name,Epoch_Start,ModelID,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
0,xTi,xTi,0.0,xTi,0.0,0.0,<QML.ModelLearningClass instance at 0x10452d758>,<DataBase.operator instance at 0x10452d680>,0.0,ReigningChampion,0.0
1,xTy,xTy,0.0,xTy,0.0,1.0,<QML.ModelLearningClass instance at 0x1045b9830>,<DataBase.operator instance at 0x1045b9ab8>,0.0,Saturated,0.0
2,z,z,0.0,zTi,0.0,3.0,<QML.ModelLearningClass instance at 0x1045b6830>,<DataBase.operator instance at 0x104644098>,0.0,ReigningChampion,1.0
3,zTi,zTi,0.0,zTi,0.0,4.0,<QML.ModelLearningClass instance at 0x10452d518>,<DataBase.operator instance at 0x104527830>,0.0,Saturated,1.0
4,yTz,yTz,0.0,yTz,0.0,5.0,<QML.ModelLearningClass instance at 0x1045db5f0>,<DataBase.operator instance at 0x104644128>,0.0,ReigningChampion,2.0
5,xPy,xPy,0.0,xTiPPyTi,0.0,6.0,<QML.ModelLearningClass instance at 0x116612710>,<DataBase.operator instance at 0x1045ff488>,0.0,Saturated,2.0
6,xTiTTx,xTiTTx,0.0,xTiTTx,0.0,7.0,<QML.ModelLearningClass instance at 0x104622b48>,<DataBase.operator instance at 0x104622998>,0.0,Active,3.0
7,xTiTTy,xTiTTy,0.0,xTiTTy,0.0,8.0,<QML.ModelLearningClass instance at 0x1046225f0>,<DataBase.operator instance at 0x1165acf80>,0.0,Active,3.0
8,xTiTTz,xTiTTz,0.0,xTiTTz,0.0,9.0,<QML.ModelLearningClass instance at 0x104622098>,<DataBase.operator instance at 0x104637710>,0.0,Active,3.0
9,zTx,zTx,0.0,zTx,0.0,10.0,<QML.ModelLearningClass instance at 0x104637b00>,<DataBase.operator instance at 0x104602998>,0.0,Active,3.0


In [28]:
qmd.runAllActiveModelsIQLE(num_exp=num_exp)

Initial time selected > 1.18902685885
Final time selected > 5.04511522418
Final Parameters mean and stdev:[ 0.47483031  0.56995576]
Initial time selected > 1.07999852347
Final time selected > 0.328806721947
Final Parameters mean and stdev:[ 0.49349828  0.57335019]
Initial time selected > 0.22337038709
Final time selected > 0.448457305131
Final Parameters mean and stdev:[ 0.50159751  0.57495053]
Initial time selected > 1.61969758942
Final time selected > 2.51177020355
Final Parameters mean and stdev:[ 0.51760872  0.57902294]
Initial time selected > 1.77984748631
Final time selected > 14.0727172941
Final Parameters mean and stdev:[ 0.48998027  0.58124545]
Initial time selected > 4.03771043228
Final time selected > 0.668957985515
Final Parameters mean and stdev:[ 0.50729044  0.57091696]
Initial time selected > 0.385831051061
Final time selected > 0.809085529969
Final Parameters mean and stdev:[ 0.52402096  0.58560491]
Initial time selected > 0.522542525198
Final time selected > 0.86521671

In [32]:
#champ_id, branch_champ_names = qmd.interBranchChampion(global_champion=True)
champ_id, branch_champ_names = qmd.interBranchChampion(branch_list=[4])

Same models, don't need to compare.
bayes_factor of log likelihoods between  xTiTTx  and  xTiTTy  =  0.358965339857
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  xTiTTz  =  0.457016611393
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  zTx  =  0.295973348546
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  zTy  =  0.740897239181
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  zTz  =  1.02681265364
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  yTzTTx  =  4.47953735837
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  yTzTTy  =  0.681044883549
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  yTzTTz  =  1.37079446191
No real winner
bayes_factor of log likelihoods between  xTiTTx  and  yTy  =  0.507128791787
No real winner
Same models, don't need to compare.
bayes_factor of log likelihoods between  xTiTTy  and  xTiTTz  =  0.24091930943
No real w

In [56]:
layerChampions

{0: []}

In [55]:
layerChampions[0] = [[]]

In [42]:
for model in list(qmd.db.loc[qmd.db['Status']=='Active']['<Name>']):
    qmd.statusChangeModel(model, new_status='Saturated')

for champ in branch_champ_names:  
    # change these ones back (change to something like 'ActiveChampion' so it doesn't run IQLE again)
    qmd.statusChangeModel(champ, new_status='ReigningChampion')

In [38]:
qmd.db

,<Name>,Alph_Name,BayesSum,Dimensionalised_Name,Epoch_Start,ModelID,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
0,xTi,xTi,0.0,xTi,0.0,0.0,<QML.ModelLearningClass instance at 0x7f7ed24d...,<DataBase.operator instance at 0x7f7ed24d1cb0>,0.0,ReigningChampion,0.0
1,xTy,xTy,0.0,xTy,0.0,1.0,<QML.ModelLearningClass instance at 0x7f7ed261...,<DataBase.operator instance at 0x7f7ed265be18>,0.0,Saturated,0.0
2,z,z,0.0,zTi,0.0,3.0,<QML.ModelLearningClass instance at 0x7f7ed24d...,<DataBase.operator instance at 0x7f7ed24d1cf8>,0.0,ReigningChampion,1.0
3,zTi,zTi,0.0,zTi,0.0,4.0,<QML.ModelLearningClass instance at 0x7f7ed24d...,<DataBase.operator instance at 0x7f7ed2779bd8>,0.0,Saturated,1.0
4,yTz,yTz,0.0,yTz,0.0,5.0,<QML.ModelLearningClass instance at 0x7f7ed276...,<DataBase.operator instance at 0x7f7ed265b950>,0.0,ReigningChampion,2.0
5,xPy,xPy,0.0,xTiPPyTi,0.0,6.0,<QML.ModelLearningClass instance at 0x7f7ed49c...,<DataBase.operator instance at 0x7f7ed26cca70>,0.0,Saturated,2.0
6,x,x,0.0,xTi,0.0,7.0,<QML.ModelLearningClass instance at 0x7f7ed276...,<DataBase.operator instance at 0x7f7ed2615440>,0.0,Saturated,3.0
7,y,y,0.0,yTi,0.0,8.0,<QML.ModelLearningClass instance at 0x7f7ed257...,<DataBase.operator instance at 0x7f7ed2761170>,0.0,ReigningChampion,3.0
8,yTx,yTx,0.0,yTx,0.0,9.0,<QML.ModelLearningClass instance at 0x7f7ed26c...,<DataBase.operator instance at 0x7f7ed2573170>,0.0,Saturated,3.0
9,zPy,yPz,0.0,zTiPPyTi,0.0,10.0,<QML.ModelLearningClass instance at 0x7f7ed49d...,<DataBase.operator instance at 0x7f7ed49b2a70>,0.0,Saturated,3.0


In [112]:
champ_id, branch_champions = qmd.interBranchChampion(global_champion=True)


Same models, don't need to compare.
bayes_factor of log likelihoods between  xTi  and  xTy  =  0.443230149081
No real winner
Same models, don't need to compare.
Champion of tree  0.0  is  xTi
Same models, don't need to compare.
bayes_factor of log likelihoods between  zTi  and  zTi  =  1.12364574871
No real winner
Same models, don't need to compare.
Champion of tree  1.0  is  z
Same models, don't need to compare.
bayes_factor of log likelihoods between  yTz  and  xTiPPyTi  =  0.407754112558
No real winner
Same models, don't need to compare.
Champion of tree  2.0  is  yTz
Same models, don't need to compare.
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.324822957984
No real winner
bayes_factor of log likelihoods between  xTi  and  yTx  =  0.0891417874579
No real winner
bayes_factor of log likelihoods between  xTi  and  zTiPPyTi  =  0.51654388741
No real winner
Same models, don't need to compare.
bayes_factor of log likelihoods between  yTi  and  yTx  =  0.302033179595
No r

sub dim =  2  num qubits :  3
Model  xTiTTx  not previously considered -- adding.
Initialization Ready
sub dim =  2  num qubits :  3
Model  xTiTTy  not previously considered -- adding.
Initialization Ready
sub dim =  2  num qubits :  3
Model  xTiTTz  not previously considered -- adding.
Initialization Ready
Model  zTx  not previously considered -- adding.
Initialization Ready
Model  zTy  not previously considered -- adding.
Initialization Ready
Model  zTz  not previously considered -- adding.
Initialization Ready
sub dim =  2  num qubits :  3
Model  yTzTTx  not previously considered -- adding.
Initialization Ready
sub dim =  2  num qubits :  3
Model  yTzTTy  not previously considered -- adding.
Initialization Ready
sub dim =  2  num qubits :  3
Model  yTzTTz  not previously considered -- adding.
Initialization Ready
Model yTx  previously considered at location 8
Model  yTy  not previously considered -- adding.
Initialization Ready
Model yTz  previously considered at location 4


## Brach generation development

In [108]:
generate_new_model_list(generator_list=['xTy', 'yTx', 'y', 'xTyTTy'])

NameError: name 'generate_new_model_list' is not defined

In [ ]:
def ising_fully_interacting(num_qubits):
    # fully interacting Ising Hamiltonian name
    this_idx = 0
    max_idx = 0
    for i in range(num_qubits):
        max_idx += i ## Triangle number of 
    p_str = ''
    for a in range(num_qubits):
        p_str += 'P'
    
    running_str=''
    for i in range(num_qubits):
        for j in range(i, num_qubits):
            if i!=j:
                this_idx+=1
                op_list = ['z', 'z']
                qub_list = [i,j]
                new_term = ModelGeneration.interaction_ham(qubit_list=qub_list, operator_list=op_list, num_qubits=num_qubits)
                running_str += new_term
                if(this_idx < max_idx):
                    running_str += p_str
    return running_str


In [7]:
ModelGeneration.random_ising_name(num_dimensions=2, num_terms=2, num_directions=2, include_identity=True)

['x', 'y', 'z', 'i']


'xTxPPzTz'

In [ ]:
ModelGeneration.

## Model comparison development

In [ ]:
treeTracker ={}
treeIDs = list(qmd.db['TreeID'].unique())

for tree in treeIDs:
    treeTracker[tree] = {}
    treeTracker[tree]['models'] = {}
    treeTracker[tree]['ModelIDs'] = list(qmd.db[qmd.db['TreeID']==tree].ModelID) #self.db[self.db['TreeID]==tree]['<Name>']
    treeTracker[tree]['Compared'] = []
    treeTracker[tree]['RankedList'] =[]
    for model in treeTracker[tree]['ModelIDs']:
        treeTracker[tree]['models'][model] = 0

In [ ]:
treeTracker[0]

In [7]:
def launch_treeDB(db):
    treeDB = pd.DataFrame({
        'TreeID' : [],
        'ModelIDs' : [],
        'RankList' : [],
        'ModelScores' : [],
        'Compared' : []
    })
    
    for treeID in list(db['TreeID'].unique()):
        populateTreeDB(db, treeDB=treeDB, treeID=treeID)    
    return treeDB

def populateTreeDB(db, treeDB, treeID):
    
    new_row  = pd.Series({
        'TreeID' : treeID,
        'ModelIDs' : list(db[db['TreeID']==treeID].ModelID),
        'RankList' : [],
        'ModelScores' : [],
        'Compared' : [] # 2^model_a_id + 2^model_b_id ?
    })
    
    num_rows = len(treeDB)
    treeDB.loc[num_rows] = new_row

    
def addModelToTreeDB(db, treeDB, model_name):
    model_tree = DataBase.pull_field(db=db, name=model_name, field='TreeID')
    model_id = DataBase.pull_field(db=db, name=model_name, field='ModelID')
    tree_list_in_db = treeDB['TreeID']
    if model_tree not in tree_list_in_db:
        populateTreeDB(db, treeDB, model_tree)
    else: 
        if model_id in list(treeDB[treeDB['TreeID']==model_tree]['ModelIDs'])[0]:
            print("Model already in treeDB")
        else:
            list(treeDB[treeDB['TreeID']==treeID]['ModelIDs'])[0].append(model_id)    

In [8]:
treeDB=launch_treeDB(qmd.db)

In [9]:
treeDB

,Compared,ModelIDs,ModelScores,RankList,TreeID
0,[],"[0.0, 1.0]",[],[],0.0
1,[],"[3.0, 4.0]",[],[],1.0
2,[],"[5.0, 6.0]",[],[],2.0
3,[],"[7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, ...",[],[],3.0


In [10]:
def compareModels(a,b, treeDB): # a, b model names
    bayes_factor = bayes_calc(a,b)
    bayes_high = 0.5 # todo 
    bayes_low = 0.5
    model_tree_a = DataBase.pull_field(db=db, name=a, field='TreeID')
    model_tree_b = DataBase.pull_field(db=db, name=b, field='TreeID')
    model_id_a = DataBase.pull_field(db=db, name=a, field='ModelID')
    model_id_b = DataBase.pull_field(db=db, name=b, field='ModelID')
    
    if model_tree_a != model_tree_b:
        print("Models compared are not in same tree.")
    
    if bayes_factor > bayes_high:
        winner = a
    elif bayes_factor < bayes_low : 
        winner = b
    else: 
        print ("No distinct winner between models ", a, "; ",b)
    
    list(treeDB[treeDB['TreeID']==model_tree]['ModelIDs'])[0].append(model_id)    
    
def bayes_factor(a,b,treeDB):
    return random.random()
    

        